In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from dataloading.loaders.openreview_loader import OpenReviewLoader

In [26]:
password = 'U^fz&*rXwXr8pyx'
username = 'zhijingshui.yang@stud.tu-darmstadt.de'
try:
    client = OpenReviewLoader(username=username, password=password)
except Exception as e:
    print(f"{e.args[0]['status']}: {str(e.args[0]['message'])}")
    print(e)

test_set = client.create_testset(ids=["zzv4Bf50RW"])

In [28]:
p = client.get_paper_reviews("zzv4Bf50RW")
p.reviews
print(p.sentences)

In [3]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('models/attitude_root/')

# Load the model
model = TFBertForSequenceClassification.from_pretrained('models/attitude_root/')

In [4]:
def predict_category(text):
    predict_input = tokenizer.encode(text,
                                    truncation=True,
                                    padding=True,
                                    return_tensors="tf")
    output = model(predict_input)[0]
    prediction_value = tf.argmax(output, axis=1).numpy()[0]
    return prediction_value

In [8]:
test_set['attitude_root_number'] = test_set['sentence'].apply(predict_category)
print(test_set)

In [ ]:
label_mapping = {
    0: 'Other',
    1: 'Clarity',
    2: 'Meaningful-comparison',
    3: 'Motivation-impact',
    4: 'Originality',
    5: 'Replicability',
    6: 'Soundness-correctness',
    7: 'Substance',
    8: 'None'
}

test_set['attitude_root'] = test_set['attitude_root_number'].map(label_mapping)

In [23]:
import pandas as pd

df = test_set
total_rows = len(df)
# Step 1: Group by 'attitude_root_number' and 'author', then aggregate the sentences
agg_df = df.groupby(['attitude_root_number', 'author']).agg(
    comments=('sentence', list), count=('sentence', 'size')
).reset_index()

agg_df


In [ ]:
final_df = agg_df.groupby('attitude_root_number').agg(
    comments=('comments', lambda x: [[author, comments] for author, comments in zip(agg_df['author'], x)]),
    count=('count', 'sum')
).reset_index()
final_df.iloc[0]['comments']

In [25]:
final_df.iloc[0]['comments']
